<a href="https://colab.research.google.com/github/Joyschool/gachon-discretemath/blob/main/12%EC%A3%BC%EC%B0%A8_%EA%B2%BD%EC%9A%B0%EC%9D%98%EC%88%98%EC%84%B8%EA%B8%B0%EC%99%80%ED%99%95%EB%A5%A0_%EC%99%84%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 10.경우의 수 세기와 확률

- (코랩) 그래프에서 한글 폰트 사용 (실행 후-> 런타임 ->세션 다시 시작)

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

- 공통 라이브러리

In [ ]:
# pydot 그래프 모듈설치(이미 설치되었으면 skip)
!pip install pydot

In [ ]:
# graphviz 그래프 모듈설치(이미 설치되었으면 skip)
!pip install graphviz

In [ ]:
# anytree - 트리를 쉽게 표현하는 라이브러리(이미 설치되었으면 skip)
!pip install anytree

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
fontname = 'NanumGothic'          # (코랩)한글 폰트
plt.rc('figure', figsize = (6,4))
plt.rc('font', family=fontname)

In [ ]:
# 트리 구조 만들기
from anytree import Node, RenderTree
from anytree.exporter import DotExporter
from IPython.display import Image, display



---



## 10-1.세기의 기본 원리

### 용어
- **세기(Counting)** : 어떤 사건이나 경우의 수를 센다
- **경우의 수(number of case)** : 일어날 수 있는 모든 사건의 수
- 경우의 수를 세는 방법
    - **합의 법칙(rule of sum)** : 두 사건 중 하나가 발생할 경우의 수 = (m + n)가지
    - **곱의 법칙(rule of product)** : 두 사건이 동시에 발생할 경우의 수 = (m x n)가지

- 길이가 n인 이진 트리 만들기

In [ ]:
from anytree import Node, RenderTree
from anytree.exporter import DotExporter
from IPython.display import Image


def generate_binary_tree(node, current_string, depth, max_depth):
    """
    조건을 만족하는 비트 문자열 트리를 생성.
    :param node: 현재 트리의 노드
    :param current_string: 현재까지의 비트 문자열
    :param depth: 현재 깊이
    :param max_depth: 최대 깊이 (비트 문자열 길이)
    """
    if depth == max_depth:
        return

    # 왼쪽 자식 노드 생성 (0 추가)
    left_child = Node(current_string + "0", parent=node, edge_label="0")
    generate_binary_tree(left_child, current_string + "0", depth + 1, max_depth)

    # 오른쪽 자식 노드 생성 (1 추가)
    right_child = Node(current_string + "1", parent=node, edge_label="1")
    generate_binary_tree(right_child, current_string + "1", depth + 1, max_depth)

def visualize_binary_tree(root, max_depth):
    """
    비트 문자열 트리를 시각화.
    :param root: 트리의 루트 노드
    :param max_depth: 최대 깊이
    """
    # 트리 생성
    generate_binary_tree(root, "", 0, max_depth)

    # 트리 출력
    print("\n조건을 만족하는 비트 문자열 트리:")
    for pre, fill, node in RenderTree(root):
        print(f"{pre}{node.name}")

    # 트리 시각화 이미지 저장 (간선에 가중치 추가)
    DotExporter(
        root,
        edgeattrfunc=lambda parent, child: f"label={child.edge_label}"  # 간선에 라벨(가중치) 추가
    ).to_picture("bit_tree_with_weights.png")
    print("\n트리가 'bit_tree_with_weights.png'로 저장되었습니다.")
    display(Image(filename="bit_tree_with_weights.png"))

# 트리의 루트 노드 생성
root = Node("root")

# 길이가 4인 비트 문자열 트리 생성 및 시각화
visualize_binary_tree(root, max_depth=4)


### [예제 10-1] 비트 문자열 경우의 수 (트리를 이용하는 방법)
연속된 두 개의 1을 갖지 않는 길이가 4인 비트 문자열은 모두 몇 개인지 구하라.

- 트리를 이용한 방법

In [ ]:
# 방법1 : 트리생성(anytree class) + 시각화(anytree)
from anytree import Node, RenderTree
from anytree.exporter import DotExporter
from IPython.display import Image


def generate_bit_tree(node, current_string, depth, max_depth):
    """
    조건을 만족하는 비트 문자열 트리를 생성.
    :param node: 현재 트리의 노드
    :param current_string: 현재까지의 비트 문자열
    :param depth: 현재 깊이
    :param max_depth: 최대 깊이 (비트 문자열 길이)
    """
    if depth == max_depth:
        return

    # 왼쪽 노드 :  '0' 추가는
    left_child = Node(current_string + "0", parent=node, edge_label="0")
    generate_bit_tree(left_child, current_string + "0", depth + 1, max_depth)

    # 오른쪽 노드 :'1' 추가는 직전 비트가 '1'이 아닐 때만 가능
    if not current_string or current_string[-1] != "1":
        right_child = Node(current_string + "1", parent=node, edge_label="1")
        generate_bit_tree(right_child, current_string + "1", depth + 1, max_depth)


def collect_leaf_nodes(root):
    """
    리프 노드의 목록과 총 개수를 반환.
    :param root: 트리의 루트 노드
    :return: (리프 노드 목록, 총 개수)
    """
    leaf_nodes = [node.name for node in root.descendants if not node.children]  # 자식이 없는 노드만 필터링
    return leaf_nodes



def visualize_tree(root):
    """
    비트 문자열 트리를 시각화.
    :param root: 트리의 루트 노드
    :param max_depth: 최대 깊이
    """

    # 트리 출력
    print("\n조건을 만족하는 비트 문자열 트리:")
    for pre, fill, node in RenderTree(root):
        print(f"{pre}{node.name}")

    # 트리 시각화 이미지 저장 (간선에 가중치 추가)
    DotExporter(
        root,
        edgeattrfunc=lambda parent, child: f"label={child.edge_label}"  # 간선에 라벨(가중치) 추가
    ).to_picture("bit_tree_with_weights.png")
    print("\n트리가 'bit_tree_with_weights.png'로 저장되었습니다.")
    display(Image(filename="bit_tree_with_weights.png"))




def count_valid_bit_strings(max_depth):
    """
    연속된 두 개의 1을 허용하지 않는 비트 문자열의 개수를 계산.
    :param max_depth: 비트 문자열의 길이
    :return: 조건을 만족하는 비트 문자열의 개수와 목록
    """
    # 트리의 루트 노드 생성
    root = Node("root")
    generate_bit_tree(root, "", 0, max_depth)

    # 리프 노드 수집
    valid_strings = collect_leaf_nodes(root)

    # anytree로 트리 시각화
    visualize_tree(root)

    return valid_strings


# 실행
if __name__ == "__main__":
    # 길이가 4인 비트 문자열 조건 확인
    bit_strings = count_valid_bit_strings(4)
    # 결과 출력
    print(f"\n조건을 만족하는 비트 문자열의 총 개수: {len(bit_strings)}")
    print(f"\n조건을 만족하는 비트 문자열: {bit_strings}")


In [ ]:
# 방법2 : 트리생성(class) + 시각화(anytree)
from anytree import Node, RenderTree
from anytree.exporter import DotExporter
from IPython.display import Image


class BinaryTree:
    def __init__(self, value=None):
        self.value = value
        self.left = None  # 0으로 확장
        self.right = None  # 1으로 확장


def generate_bit_tree(root, depth, max_depth):
    """
    연속된 두 개의 1을 허용하지 않는 이진 트리를 생성.
    :param root: 현재 노드
    :param depth: 현재 깊이
    :param max_depth: 트리의 최대 깊이 (문자열의 길이)
    """
    if depth == max_depth:
        return

    # 왼쪽 노드 :  ('0' 추가는 항상 가능)
    root.left = BinaryTree(root.value + "0" if root.value else "0")
    generate_bit_tree(root.left, depth + 1, max_depth)

    # 오른쪽 노드 : ('1' 추가는 직전 비트가 '1'이 아닐 때만 가능)
    if not root.value or root.value[-1] != "1":
        root.right = BinaryTree(root.value + "1" if root.value else "1")
        generate_bit_tree(root.right, depth + 1, max_depth)


def convert_to_anytree(root, anytree_parent=None):
    """
    BinaryTree를 anytree 구조로 변환.
    :param root: BinaryTree의 루트 노드
    :param anytree_parent: anytree의 부모 노드
    :return: 변환된 anytree 노드
    """
    if root is None:
        return None

    # 현재 노드를 anytree 노드로 생성
    anytree_node = Node(root.value if root.value else "root", parent=anytree_parent)

    # 왼쪽 및 오른쪽 자식을 재귀적으로 변환
    convert_to_anytree(root.left, anytree_node)
    convert_to_anytree(root.right, anytree_node)

    return anytree_node


def visualize_to_anytree(root):
    """
    anytree를 사용하여 트리를 시각화.
    :param binary_root: BinaryTree의 루트 노드
    """
    # BinaryTree를 anytree 구조로 변환
    anytree_root = convert_to_anytree(root)

    # # 텍스트 기반 트리 출력
    # print("\n트리 구조 (텍스트 기반):")
    # for pre, fill, node in RenderTree(anytree_root):
    #     print(f"{pre}{node.name}")

    # Graphviz를 사용한 트리 시각화 (이미지 저장)
    DotExporter(anytree_root).to_picture("binary_tree.png")
    print("\n트리가 'binary_tree.png'로 저장되었습니다.")
    display(Image(filename="binary_tree.png"))


def collect_leaf_nodes(root):
    """
    리프 노드를 수집하여 비트 문자열을 반환.
    :param binary_root: 트리의 루트 노드
    :return: 리프 노드 값의 리스트
    """
    if root is None:
        return []

    # 리프 노드일 경우 값을 반환
    if root.left is None and root.right is None:
        return [root.value]

    # 왼쪽과 오른쪽의 리프 노드를 재귀적으로 수집
    return collect_leaf_nodes(root.left) + collect_leaf_nodes(root.right)


def count_valid_bit_strings(max_depth):
    """
    연속된 두 개의 1을 허용하지 않는 비트 문자열의 개수를 계산.
    :param max_depth: 비트 문자열의 길이
    :return: 조건을 만족하는 비트 문자열의 개수와 목록
    """
    # BinaryTree 트리 생성
    root = BinaryTree()
    generate_bit_tree(root, 0, max_depth)

    # 리프 노드 수집
    valid_strings = collect_leaf_nodes(root)

    # anytree로 트리 시각화
    visualize_to_anytree(root)

    return valid_strings


# 실행
if __name__ == "__main__":
    # 길이가 4인 비트 문자열 조건 확인
    bit_strings = count_valid_bit_strings(4)
    # 결과 출력
    print(f"\n조건을 만족하는 비트 문자열의 총 개수: {len(bit_strings)}")
    print(f"\n조건을 만족하는 비트 문자열: {bit_strings}")


- 점화식을 이용한 방법
- $f(n) = f(n-1) + f(n-2)$
- $ f(1) = 2$ (0, 1)
- $ f(2) = 3$ (00,01,10)

In [ ]:
def count_valid_bit_strings(n):
    if n == 1:
        return 2  # 0, 1
    if n == 2:
        return 3  # 00, 01, 10

    # 점화식을 이용한 피보나치 방식
    a, b = 2, 3
    for _ in range(3, n + 1):
        a, b = b, a + b
    return b

# 길이가 4인 비트 문자열 계산
print("길이가 4인 비트 문자열 중 조건을 만족하는 개수:", count_valid_bit_strings(4))


In [ ]:
def count_valid_bit_strings(n):
    if n == 1:
        return 2  # 0, 1
    if n == 2:
        return 3  # 00, 01, 10

    return count_valid_bit_strings(n-1)+count_valid_bit_strings(n-2)

# 길이가 4인 비트 문자열 계산
print("길이가 4인 비트 문자열 중 조건을 만족하는 개수:", count_valid_bit_strings(4))

### [예제 10-2] 주사위 합이 홀수인 경우의 수(표를 이용하여 구하는 방법()
두 개의 주사위 A, B를 동시에 던졌을 때 주사위의 눈의 합이 홀수가 나오는 경우의 수를 구하는

In [ ]:
def count_odd_sum_cases():
    # 경우의 수 계산
    cases = 0
    numbers = ''
    for a in range(1, 7):  # 주사위 A의 눈
        for b in range(1, 7):  # 주사위 B의 눈
            if (a + b) % 2 == 1:  # 합이 홀수인 경우
                cases += 1
                print(a, b, end=' | ')

    return cases

# 결과 출력
result = count_odd_sum_cases()
print(f"주사위의 눈의 합이 홀수가 나오는 경우의 수: {result}")


### [예제 10-3] 강좌 선택 경우의 수(합의 법칙)
학과 수업이 오전에 4 강좌, 오후에 6강좌를 개설하는 학과가 있다고 하자. 개설된 강좌 중에서 한 강좌만 수강한다고 할 때, 선택 방법이 몇 가지인지 구하라.

In [ ]:
def factorial(n):
    if n == 0 or n == 1:
        return 1
    return n * factorial(n - 1)


def comb(n, r):
    return factorial(n) // (factorial(r) * factorial(n - r))


def count_selection_methods(total):
    # 1. 한 강좌만 수강
    one_course = comb(total, 1)

    # 2. 두 강좌를 수강
    two_courses = comb(total, 2)

    return one_course, two_courses

# 결과 계산
one_course, two_courses = count_selection_methods(4 + 6)
print(f"1. 한 강좌만 수강하는 경우의 수: {one_course}")
print(f"2. 두 강좌를 수강하는 경우의 수: {two_courses}")


In [ ]:
from math import comb

def count_selection_methods(total):
    # 1. 한 강좌만 수강
    one_course = comb(total, 1)

    # 2. 두 강좌를 수강
    two_courses = comb(total, 2)

    return one_course, two_courses

# 결과 계산
one_course, two_courses = count_selection_methods(4 + 6)
print(f"1. 한 강좌만 수강하는 경우의 수: {one_course}")
print(f"2. 두 강좌를 수강하는 경우의 수: {two_courses}")

In [ ]:
from itertools import combinations

def count_selection_methods(total):
    # 1. 한 강좌만 수강
    one_course = len(list(combinations(range(total), 1)))

    # 2. 두 강좌를 수강
    two_courses = len(list(combinations(range(total), 2)))

    return one_course, two_courses

# 결과 계산
one_course, two_courses = count_selection_methods(4 + 6)
print(f"1. 한 강좌만 수강하는 경우의 수: {one_course}")
print(f"2. 두 강좌를 수강하는 경우의 수: {two_courses}")


### [예제 10-4] 임원 선출 경우의 수(곱의 법칙: 순열)
회원이 32명인 어느 대학교 동아리에서 회장, 부회장, 총무를 뽑을 수 있는 방법이 몇 가지인지 구하라. 단, 어느 한 사람이 1개 이상의 직위에 뽑힐 수 없다.

In [ ]:
from math import perm

def count_permutations(n, r):
    """
    n명의 회원 중에서 r개의 직위를 뽑는 순열의 경우의 수를 계산.
    """
    return perm(n, r)

# 결과 계산
n = 32  # 회원 수
r = 3   # 직위 개수
result = count_permutations(n, r)
print(f"회원 32명 중 회장, 부회장, 총무를 뽑는 경우의 수: {result}")


In [ ]:
from itertools import permutations

def count_permutations_itertools(n, r):
    """
    n명의 회원 중에서 r개의 직위를 뽑는 순열의 경우의 수를 계산.
    """
    return len(list(permutations(range(n), r)))

# 결과 계산
n = 32  # 회원 수
r = 3   # 직위 개수
result = count_permutations_itertools(n, r)
print(f"회원 32명 중 회장, 부회장, 총무를 뽑는 경우의 수: {result}")


### [예제 10-5] 문자열 식별자 경우의 수
어떤 프로그래밍 언어에서 식별자identifier 이름은 영어 알파벳 소문자 하나로 구성되거나, 영어 알파벳 소문자로 시작해서 두 번째와 세번째 기호는 숫자 혹은 영어 알파벳 소문자가 될 수 있는 길이가 최대 3인 문자열로 구성된다. 이 언어에서 만들 수 있는 식별자 이름의 가짓수를 구하라.  

In [ ]:
def count_identifiers():
    # 길이 1: 영어 소문자 (26가지)
    length_1 = 26

    # 길이 2: 첫 문자는 영어 소문자, 두 번째는 영어 소문자 또는 숫자 (26 + 10 = 36)
    length_2 = 26 * 36

    # 길이 3: 첫 문자는 영어 소문자, 두 번째와 세 번째는 영어 소문자 또는 숫자
    length_3 = 26 * 36 * 36

    # 총 가짓수
    total_identifiers = length_1 + length_2 + length_3

    return total_identifiers

# 결과 출력
total = count_identifiers()
print(f"이 언어에서 만들 수 있는 식별자 이름의 총 가짓수: {total}")


### @비둘기 집의 원리(pigeonhole principle; 분류함 원칙)
𝒏개의 비둘기 집이 있고, 𝒎마리의 비둘기가 있을 때, 𝒎 > 𝒏이라고 하면 𝒎마리의 비둘기를 𝒏개의 비둘기 집에 넣을 때 2마리 이상의 비둘기가 들어가는 비둘기 집이 적어도 하나 이상 있다

### [예제 10-7] 비둘기 집의 원리 예제1
만약 서로 다른 5개의 숫자가 1부터 8 사이에서 선택될 때, 그들 중 2개의 숫자는 더해서 9가 됨을 보여라.      

In [ ]:
from itertools import combinations

def has_pair_with_sum(numbers, target_sum):
    """
    주어진 숫자 집합에서 두 숫자의 합이 특정 값이 되는지 확인.
    """
    # 두 숫자의 조합을 생성
    for pair in combinations(numbers, 2):
        if sum(pair) == target_sum:
            print(f"합이 {target_sum}인 쌍: {pair}")
            return True
    return False

# 입력 값
numbers = [1, 2, 3, 4, 5]  # 1부터 8 중 5개 선택
target_sum = 9

# 결과 확인
if has_pair_with_sum(numbers, target_sum):
    print(f"주어진 숫자 {numbers} 중 합이 {target_sum}인 쌍이 존재합니다.")
else:
    print(f"주어진 숫자 {numbers} 중 합이 {target_sum}인 쌍이 없습니다.")


### [예제 10-7] 비둘기 집의 원리 예제2
가로, 세로 길이가 2m인 방안에 5개의 인형을 놓을 때, √2 m 이내에 두 인형이 항상 있게 됨을 보여라

In [ ]:
from itertools import combinations
import math
import random

def calculate_distance(point1, point2):
    """
    두 점 사이의 거리를 계산.
    """
    return math.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

def check_minimum_distance(points, threshold):
    """
    주어진 점들 중 최소 두 점 사이의 거리가 threshold(임계값) 이내인지 확인.
    """
    for point1, point2 in combinations(points, 2):
        distance = calculate_distance(point1, point2)
        if distance <= threshold:
            print(f"\n거리 {threshold} m 이내에 있는 두 인형 ")
            print(f"- 위치: {point1}, {point2}")
            print(f"- 거리: {distance}")
            return True
    return False

def random_placement_simulation(num_points=5, room_size=2):
    """
    방 안에 무작위로 인형을 배치하고 조건을 확인하는 시뮬레이션.
    :param num_points: 인형의 개수
    :param room_size: 방의 크기 (정사각형 방의 한 변 길이)
    """
    # 방의 크기 내에서 인형을 랜덤 배치
    # - random.uniform: 두 수 사이의 실수 중에서 난수값을 리턴
    points = []
    for _ in range(num_points):
        point = (random.uniform(0, room_size), random.uniform(0, room_size))
        points.append(point)
        print(f"인형 좌표: {point}")

    # 거리 조건 확인
    if check_minimum_distance(points, math.sqrt(2)):
        print("\n√2 m 이내에 항상 두 인형이 있습니다.")
    else:
        print("\n조건을 만족하지 않는 배치가 발생했습니다.")  # 이론적으로 불가능

# 실행
random_placement_simulation()


In [ ]:
100/12

--------------------------------

## 10-2.순열과 조합

- https://docs.python.org/ko/3/library/itertools.html

### @중복순열(r-sample) : 중복 O, 순서 O
- 𝑛개의 원소로부터 𝑟개를 추출하는 가짓수
> **$𝑛×𝑛× ⋯ ×𝑛 = n^r $가지**

### [예제 10-9] 중복 순열



In [ ]:
import itertools

S = {1, 2, 3}
R = list(itertools.product(S, repeat=2))

print(f'중복 순열 경우의 수: {len(R)}')
R

### @중복조합(r-selection) : 중복 O, 순서 X
- 𝑛개의 원소로부터 𝑟개를 추출하는 가짓수
> $_{n}H_r$ = $_{n+r-1}C_r$ = $\frac{(n+r-1)!}{r!(n-1)!}$

### [예제 10-11] 중복 조합

In [ ]:
S = {1, 2, 3}
R = list(itertools.combinations_with_replacement(S, 2))

print(f'중복 조합 경우의 수: {len(R)}')
R

### @순열(r-permutation) : 중복 X, 순서 O
- 𝑛개의 원소로부터 𝑟개를 추출하는 가짓수
> $_{n}P_r$ = $\frac{n!}{(n-r)!}$

### [예제 10-12] 순열

In [ ]:
S = {1, 2, 3}
R = list(itertools.permutations(S, 2))

print(f'순열 경우의 수: {len(R)}')
R

### @팩토리얼(factorial) 표기법
- $ 𝑛! = 1×2×3×4×⋯×(𝑛−2)×(𝑛−1)×𝑛 $
- 재귀법(recursion)에 의해
>$𝑛! = 𝑛×(𝑛−1)! ,  0! =1,  1! = 1$이다.

In [ ]:
# 단순 반복 함수로 구현
def factorial_for(n):
    ret = 1
    for i in range(1, n+1):
        ret *= i  # ret = ret * i
    return ret

factorial_for(5)

In [ ]:
# 재귀함수 사용
def factorial_recursive(n):
    if n > 1 :
        return n * factorial_recursive(n-1)
    else :
        return 1

factorial_recursive(5)

In [ ]:
# 재귀함수 사용
def factorial_recursive(n):
    return n * factorial_recursive(n-1) if n > 1 else 1

factorial_recursive(5)

In [ ]:
# 이미 만들어진 함수 사용 : math.factorial
import math

math.factorial(5)

In [ ]:
# # 이미 만들어진 함수 사용 : functools.reduce
# reduce: 원 자료구조의 부분구조를 반복적으로 처리해 재결합해서 하나의 결과값으로 반환하는 고순위(higher-order) 함수
from functools import reduce

def factorial_reduce(n):
    return reduce(lambda x, y: x * y, range(1, n+1))

### [예제 10-13] 순열
문자 c, a, b, i, n, e, t로부터 4개의 문자로 구성된 단어를 몇 개나 만들 수 있는지 구하라.

In [ ]:
S = {'c','a','b','i','n','e','t'}
R = list(itertools.permutations(S, 4))
# R = list(itertools.permutations('cabinet', 4))

print(f'순열 경우의 수: {len(R)}')

### @조합(r-combination) : 중복 X, 순서 X
- 𝑛개의 원소로부터 𝑟개를 추출하는 가짓수
> $_{n}C_r$ = $\frac{n!}{r!(n-r)!}$

### [예제 10-14] 조합

In [ ]:
S = {1, 2, 3}
R = list(itertools.combinations(S, 2))

print(f'조합 경우의 수: {len(R)}')
R

### [예제 10-15] 조합
중국 음식점에서 8개의 접시에 서로 다른 음식이 있다. 서로 다른 두 가지의 음식을 취할 수 있는 가짓수를 구하라.

In [ ]:
S = list(range(1,9)) # 중국 음식 8개
R = list(itertools.combinations(S, 2))

print(f'음식을 취할 수 있는 조합 경우의 수: {len(R)}')
R

### [예제 10-16] 조합
포커 게임에서 한 사람이 받는 5장 카드의 조합은 몇 가지인지 계산하라

In [ ]:
S = list(range(1,53))  # 포커 게임 카드 개수
R = list(itertools.combinations(S, 5))

print(f'한 사람이 받는 카드의 조합 경우의 수: {len(R):,}')

### [ 포커 게임]  일부 기능(틀리플, 플러시, 로얄 플러시)만 구현함

In [ ]:
import random

class PokerHandSimulator():

    # 카드 만들기
    def __init__(self, n):
        shapes = '◇♡♤♧'
        nums = [2,3,4,5,6,7,8,9,10,'J','Q','K','A']
        self.deck_of_cards = []  # 카드 담을 리스트
        self.n = n               # 시행 횟수
        for shape in shapes:
            for num in nums:
                self.deck_of_cards.append(shape+str(num))
        print(f'deck_of_cards : 총{len(self.deck_of_cards)}개')
        print(f'{self.deck_of_cards}')

        self.log = {}  # 카드의 결과를 담을 딕셔너리

    # 카드의 결과를 기록하기
    def write_log(self, result):
        if result not in self.log.keys(): # 동일한 종류의 결과가 없으면
            self.log[result] = 1
        else:                             # 동일한 종류의 결과가 있으면
            self.log[result] += 1

    # 카드 나눠주기
    def main(self):
        # self.n 만큼 반복하기
        for i in range(self.n):
            random.shuffle(self.deck_of_cards) # 카드 섞기

            # deck_of_cards에서 맨 위의 5개 카드가져오기
            my_cards = self.deck_of_cards[:5]
            print(f'[{i+1:>5}] my_cards: {my_cards}')

            # 내 카드 조합 확인하기
            self.check_hand(my_cards)

        # 확률 구하기 ------------
        print('-'*50)
        print(f'#Total simulations: {self.n} times')
        for k, v in self.log.items():
            print(f'- {k} : {v} ({v/self.n*100:>.2f}%)')


    # 카드 조합 확인하기
    def check_hand(self, cards):
        result = 'NONE'
        if self.is_tree_of_a_kind(cards):     # 트리플
            result = self.is_tree_of_a_kind(cards)
        elif self.is_royal_flush(cards):      # 로얄플러시
            result = self.is_royal_flush(cards)
        elif self.is_flush(cards):            # 플러시
            result = self.is_flush(cards)

        self.write_log(result)       # 카드의 결과 기록하기

        print(f'result: {result}')


    # 카드의 종류가 트리플(tree_of_a_kind)인지 확인
    def is_tree_of_a_kind(self, cards):
        nums = [c[1:] for c in cards]  # 카드들의 숫자만 담기
        set_nums = list(set(nums))     # 중복안된 숫자 종류만 추출
        for num in set_nums:
            if nums.count(num) == 3:
                return 'Three Of A Kind'
            else:
                return False

    # 카드의 종류가 플러시(flush)인지 확인
    def is_flush(self, cards):
        suits = [c[0] for c in cards]  # 카드의 종류만 추출
        if len(set(suits)) == 1:       # 모든 카드의 종류가 1가지
            return 'Flush'
        else:
            return False

    # 카드의 종류가 로얄 플러시(royal_flush)인지 확인
    def is_royal_flush(self, cards):
        suits = [c[0]  for c in cards]  # 카드의 종류만 추출
        nums  = [c[1:] for c in cards]  # 카드들의 숫자만 담기

        # 로얄 플러시(=10,J,Q,K,A) and 모든 카드의 종류가 1가지
        if len(set(suits)) == 1 and (sorted(nums) == sorted(['10','J','Q','K','A'])):
            return 'Royal Flush'
        else:
            return False


In [ ]:
N = int(input('#--포커 확률 시뮬레이터--\n몇번을 실행할까요?: '))
app = PokerHandSimulator(N)

# 방법1: 임의로 카드 부여
app.main()  # 임의로 카드 부여하기

# 방법2: 테스트를 위한 방법
# my_cards = ['♧3', '♡10', '♤10', '◇10', '♡2']
# my_cards = ['♡3', '♡10', '♡K', '♡9', '♡2']
# my_cards = ['♡A', '♡10', '♡K', '♡J', '♡Q']
# print(f'my_cards : {my_cards}')
# app.check_hand(my_cards)

--------------

## 10-3.이항정리와 다항정리

### @이항정리
- $(x+y)^n$ = $\sum_{r=0}^n$ $_nC_rx^{n-r}y^r$
- 일반항 $_nC_rx^{n-r}y^r$

### [예제 10-22] 이항계수 구하기
$(3x-y)^5$을 전개했을 때, $x^2y^3$의 계수를 구하라.

In [ ]:
from math import comb

def binominal_coefficient(a, b, n, x_exp, y_exp):
    """
    이항식 전개에서 특정 항의 계수를 계산하는 함수
    """
    # y의 지수는 r, x의 지수는 n-r
    r = y_exp
    if x_exp != n - r:
        return 0  # 주어진 지수 조건이 맞지 않으면 0 반환

    # 계수 계산 :
    return comb(n, r) * (a ** (n - r)) * (b ** r)

# 문제에 맞는 값 설정
a = 2    # x의 계수
b = -1   # y의 계수
n = 5    # 전체 다항식의 지수
x_exp = 2  # x의 지수
y_exp = 3  # y의 지수

# 계수 계산
coefficient = binominal_coefficient(a, b, n, x_exp, y_exp)

# 결과 출력
print(f"(3x - y)^5에서 x^2y^3의 계수는: {coefficient}")


### @파스칼 삼각형
이항계수를 삼각형 형태로 나타낸 것

In [ ]:
def pascals_triangle(n):
    """
    파스칼의 삼각형을 출력하는 함수
    :param n: 삼각형의 행 수
    """
    triangle = []  # 파스칼의 삼각형 저장

    for i in range(n):
        row = [1]  # 각 행의 첫 번째 항은 항상 1
        for j in range(1, i + 1):
            # 이항 계수 계산: 이전 행을 기반으로 현재 값을 계산
            value = triangle[i - 1][j - 1] + triangle[i - 1][j] if j < len(triangle[i - 1]) else 1
            row.append(value)
        triangle.append(row)


    # 최대 폭 계산 (중심 정렬에 사용)
    max_width = len(" ".join(map(str, triangle[-1])))

    # 출력
    for row in triangle:
        print(" ".join(map(str, row)).center(max_width))


# 파스칼의 삼각형 출력
rows = 10  # 원하는 행 수
pascals_triangle(rows)


### @다항정리
- 두 개 이상의 항을 가진 다항식$(x_1+x_2+...+x_n)^n$을 전개할 때 사용하는 수학적 정리
- 이항정리 $(a+b)^n$의 확장판.  k개의 항의 값을 계산할 수 있음
- 다항정리 공식
> $(x_1+x_2+...+x_n)^n$ = $\sum_{r_1+r_2+...+r_k=n}$ $\frac{n!}{r_1! r_2! ... r_k!}$ $x_{1}^{r_1}x_{2}^{r_2}...x_{k}^{r_k}$
- 다항계수 : $\frac{n!}{r_1! r_2! ... r_k!}$

### [예제 10-23] 다항정리 : 계수 구하기
$(x+y+z)^9$를 전개했을 때 $x^2y^3z^4$의 계수를 구하라

In [ ]:
from math import factorial

def multinomial_coefficient(n, ks):
    """
    다항 계수를 계산하는 함수.
    """
    # n! / (k1! * k2! * k3! ...)
    return factorial(n) // (factorial(ks[0]) * factorial(ks[1]) * factorial(ks[2]))

# 입력값 설정
n = 9                 # 전체 차수
k1, k2, k3 = 2, 3, 4  # x^2, y^3, z^4

# 다항 계수 계산
coefficient = multinomial_coefficient(n, [k1, k2, k3])

# 결과 출력
print(f"(x + y + z)^9에서 x^2y^3z^4의 계수는: {coefficient}")


------------------------------------------

## 10-4.확률의 기초 및 조건부 확률


### @확률(Probability)의 성질
표본공간 S의 각 사건 $A_i$가 일어날 확률을 $P(A_i)$라고 할 때
- (1) 어떤 사건이 일어날 확률은 다음과 같다.
> $0 \le P(A) \le 1$ <br>
- (2) 한 표본공간의 모든 사건들의 확률의 합은 1이다.
> $\sum_{i=1}^{n} P(A) = 1$

### @배반사건(exclusive events)
- 2개의 사건 𝑨, 𝑩가 동시에 일어날 수 없을 때, 즉 한쪽이 일어나면 다른 쪽이 일어나지 않을 때의 두 사건을 말한다. (배반사건은 뒤에서 설명하는 독립 사건과는 다르다.)
- 배반사건의 합의 법칙
> $P(A \cup B) = P(A) + P(B) $
- 배반사건이 아닐 때의 합의 법칙
> $P(A \cup B) = P(A) + P(B) -  P(A \cap B)$



### @조건부 확률(conditional_probability)
표본공간의 선정에 따라 확률이 달라지는 것을 고려한 확률
- "B가 주어졌을 때 𝐴가 일어날 확률"이다
> $P(A|B) = \frac{P(A \cap B)} {P(B)}$

### 조건부 확률 계산

In [ ]:
import pandas as pd

tiresale = {'M' : {'G':64, 'B':16},
            'P' : {'G':42, 'B':78}}

table = pd.DataFrame(tiresale).transpose()
table

In [ ]:
# 각각 확률
P_M = table.loc['M'].sum() / table.values.sum()  # 유명 메이커가 선택되는 사건
P_P = table.loc['P'].sum() / table.values.sum()  # 사제품이 선택되는 사건
P_G = table['G'].sum() / table.values.sum()      # 양질의 서비스가 선택되는 사건
P_B = table['G'].sum() / table.values.sum()
P_G_M = table.loc['M']['G'] / table.loc['M'].sum()
P_B_M = table.loc['M']['B'] / table.loc['M'].sum()
P_G_P = table.loc['P']['G'] / table.loc['P'].sum()
P_B_P = table.loc['P']['B'] / table.loc['P'].sum()
print(f'P_M(메이커): {P_M}')
print(f'P_P(시제품): {P_P}')
print(f'P_G(양질의서비스): {P_G}')
print(f'P_B(저질의서비스): {P_B}')
print(f'P_G_M(메이커면서 양질의서비스): {P_G_M}')
print(f'P_B_M: {P_B_M}')
print(f'P_G_P: {P_G_P}')
print(f'P_B_P: {P_B_P}')

### [예제 10-29] 조건부 확률
어떤 학급에서 남학생일 확률 P(A)=0.54이고, 안경 쓸 확률 P(B)=0.81일 때, 안경 쓴 남학생일 확률 P(A⋂B)=0.18이다. 남학생인 경우에 안경 쓸 확률과 안경 쓴 학생인 경우 남학생일 확률을 구하라.  

In [ ]:
def conditional_probabilities(P_A, P_B, P_A_and_B):
    """
    조건부 확률을 계산하는 함수
    :param P_A: 남학생일 확률 (P(A))
    :param P_B: 안경 쓸 확률 (P(B))
    :param P_A_and_B: 안경 쓴 남학생일 확률 (P(A ∩ B))
    :return: (P(B|A), P(A|B)) - 남학생인 경우 안경 쓸 확률, 안경 쓴 경우 남학생일 확률
    """
    # 조건부 확률 계산
    P_B_given_A = P_A_and_B / P_A if P_A != 0 else 0
    P_A_given_B = P_A_and_B / P_B if P_B != 0 else 0

    return P_B_given_A, P_A_given_B

# 입력 값
P_A = 0.54   # 남학생일 확률
P_B = 0.81   # 안경 쓸 확률
P_A_and_B = 0.18  # 안경 쓴 남학생일 확률

# 함수 호출
P_B_given_A, P_A_given_B = conditional_probabilities(P_A, P_B, P_A_and_B)

# 결과 출력
print(f"1. 남학생인 경우에 안경 쓸 확률 P(B|A): {P_B_given_A:.2f}")
print(f"2. 안경 쓴 학생인 경우 남학생일 확률 P(A|B): {P_A_given_B:.2f}")


### 응용: 조건부 확률 계산

In [ ]:
from numpy import random

random.seed(0)

N = 100000

연령층 = [20, 30, 40, 50, 60, 70]
연령층인구수 = {age:0 for age in 연령층}
연령층구매량 = {age:0 for age in 연령층}
전체구매량 = 0

for _ in range(N):
    특정연령층 = random.choice(연령층)
    구매확률 = float(특정연령층) / 100.0
    연령층인구수[특정연령층] += 1
    if random.random() < 구매확률:
        전체구매량 += 1
        연령층구매량[특정연령층] += 1


for age in 연령층:
    print(f'{age}대 인구비율: {float(연령층인구수[age]/N)}')
    print(f'{age}대 구매확률: {float(연령층구매량[age]/N)}')
    print(f'{age}대 구매할 조건부확룰: {float(연령층구매량[age]/N) / float(연령층인구수[age]/N)}')
    print('-'*30)

print(f'인구비율: {sum(연령층인구수.values())/N}')

### [예제 10-31]
- (a) 동전을 두 개 던져서 모두 앞면이 나올 때의 확률
- (b) 52장의 카드에서 2장을 뽑을 때 첫 번째 카드를 뽑고 다시 넣은 후
두 번째 카드를 뽑아서 두 장 모두 에이스일 확률
- (c) 52장의 카드에서 2장을 뽑을 때, 첫 번째 카드를 뽑고 다시 넣지 않은 후 두 번째 카드를 뽑아서 2장 모두 에이스일 확률  

In [ ]:
from itertools import product

def probability_two_heads():
    # 가능한 결과: 'H' (앞면), 'T' (뒷면)
    outcomes = list(product(['H', 'T'], repeat=2))

    # 성공 케이스: ('H', 'H')
    success = [outcome for outcome in outcomes if outcome == ('H', 'H')]

    # 확률 계산
    probability = len(success) / len(outcomes)
    return probability

# 결과 출력
print("(a) 동전을 두 개 던져 모두 앞면이 나올 확률:", probability_two_heads())


In [ ]:
def probability_two_aces_replacement():
    # 각 뽑기에서 가능한 카드: 'Ace' (4장) + 'Other' (48장)
    deck = ['Ace'] * 4 + ['Other'] * 48

    # 두 번 뽑기 (복원추출)
    outcomes = list(product(deck, repeat=2))

    # 성공 케이스: ('Ace', 'Ace')
    success = [outcome for outcome in outcomes if outcome == ('Ace', 'Ace')]

    # 확률 계산
    probability = len(success) / len(outcomes)
    return probability

# 결과 출력
print("(b) 두 장 모두 에이스 (복원추출):", probability_two_aces_replacement())


In [ ]:
from itertools import permutations

def probability_two_aces_no_replacement():
    # 덱 생성
    deck = ['Ace'] * 4 + ['Other'] * 48

    # 두 장 뽑기 (비복원추출)
    outcomes = list(permutations(deck, 2))

    # 성공 케이스: 첫 번째와 두 번째 카드가 모두 'Ace'
    success = [outcome for outcome in outcomes if outcome[0] == 'Ace' and outcome[1] == 'Ace']

    # 확률 계산
    probability = len(success) / len(outcomes)
    return probability

# 결과 출력
print("(c) 두 장 모두 에이스 (비복원추출):", probability_two_aces_no_replacement())


---------------------

THE END